# Expand reader group coverage
In prior analysis, we have seen that some reader groups have poor coverage, specifically expertise and demographics.

Let's use the prior author data to expand this coverage.

In [4]:
import pandas as pd
from importlib import reload
import data_helpers
reload(data_helpers)
from data_helpers import load_all_author_data
author_data_dir = '../../data/reddit_data/author_data/'
usecols = ['author', 'subreddit', 'created_utc', 'id']
author_data = load_all_author_data(author_data_dir, usecols=usecols)
print(author_data.loc[:, 'author'].nunique())

  3%|▎         | 7097/204619 [00:19<08:45, 375.81it/s]

skipping file ../../data/reddit_data/author_data/OJnToothpaste_comments.gz because error Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.



 13%|█▎        | 26705/204619 [04:13<3:55:08, 12.61it/s] 

skipping file ../../data/reddit_data/author_data/papercut_eyelid_comments.gz because error Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.



 30%|██▉       | 60381/204619 [16:16<31:27, 76.42it/s]   

skipping file ../../data/reddit_data/author_data/Quovef_comments.gz because error Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.



 69%|██████▉   | 141185/204619 [44:10<14:22, 73.53it/s]   

skipping file ../../data/reddit_data/author_data/cypher_chyk_comments.gz because error Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.



100%|██████████| 204619/204619 [1:06:21<00:00, 51.39it/s]  


189139


In [6]:
## limit to actual question askers
question_comment_data = pd.read_csv('../../data/reddit_data/advice_subreddit_no_filter_comment_question_data.gz', sep='\t', compression='gzip', usecols=['author', 'parent_id', 'subreddit', 'created_utc'])

In [14]:
date_bins

[datetime.datetime(2018, 1, 1, 0, 0),
 datetime.datetime(2018, 7, 1, 0, 0),
 datetime.datetime(2019, 1, 1, 0, 0),
 datetime.datetime(2019, 7, 1, 0, 0)]

In [22]:
## get date info
from datetime import datetime, timedelta
from monthdelta import monthdelta
import re
# num_matcher = re.compile('\d+\.\d')
# author_data = author_data.assign(**{
#     'date' : author_data.loc[:, 'created_utc'].apply(lambda x: datetime.fromtimestamp(int(float(x))) if (type(x) is str and num_matcher.match(x) is not None) else x),
# })
# question_comment_data = question_comment_data.assign(**{
#     'date' : question_comment_data.loc[:, 'created_utc'].apply(lambda x: datetime.fromtimestamp(int(float(x)))),
# })
## convert to date-day bins
from math import ceil
import numpy as np
start_year, end_year = 2018, 2019
start_month, end_month = 1, 12
bin_month_width = 6
months_per_year = 12
date_bin_count = int(ceil(((end_year - start_year + 1) * months_per_year) / bin_month_width))
start_date = datetime(year=start_year, month=1, day=1)
date_bins = [start_date + monthdelta(bin_month_width*i) for i in range(date_bin_count)]
date_bins = np.array(list(map(lambda x: x.timestamp(), date_bins)))
from data_helpers import assign_date_bin
author_data = author_data.assign(**{
    'date_day_bin' : author_data.loc[:, 'date'].apply(lambda x: assign_date_bin(x, date_bins) if (type(x) is float and not np.isnan(x)) else None),
})
question_comment_data = question_comment_data.assign(**{
    'date_day_bin' : question_comment_data.loc[:, 'date'].apply(lambda x: assign_date_bin(x.timestamp(), date_bins))
})

In [23]:
author_data.loc[:, 'date_day_bin'].unique()

array([datetime.datetime(2019, 7, 1, 0, 0),
       datetime.datetime(2019, 1, 1, 0, 0),
       datetime.datetime(2018, 7, 1, 0, 0),
       datetime.datetime(2018, 1, 1, 0, 0), -1, None], dtype=object)

## Expand expertise
Let's expand the definition of "expert" by counting the prior posts in topic-relevant subreddits.

Author $a$ posts a question in subreddit $s$.
1. $a$ is an "expert" if at least 90% of their prior posts are in subreddit $s$.
2. $a$ is an "expert" if at least 90% of their prior posts are in subreddits $\mathcal{S}_{t}$, i.e. $s$ or one of its top-k neighbors at time $t$.

In [53]:
## connect authors with original question time + subreddit info
question_comment_data.rename(columns={'subreddit' : 'question_subreddit', 'date_day_bin' : 'question_date_day_bin'}, inplace=True)
author_question_data = pd.merge(author_data, question_comment_data.loc[:, ['author', 'question_subreddit', 'question_date_day_bin']], on='author', how='right')
# fix data type
author_question_data = author_question_data.assign(**{
    'question_date_day_bin' : pd.Series(author_question_data.loc[:, 'question_date_day_bin'].values, dtype='object')
})
# remove null data
author_question_data = author_question_data[author_question_data.loc[:, 'date_day_bin'].apply(lambda x: type(x) is datetime)]

In [39]:
print(author_question_data.shape[0])
display(author_question_data.head())

69616031


,author,subreddit,created_utc,id,date,date_day_bin,question_subreddit,question_date_day_bin
0,DaveAlot,ultrawidemasterrace,1617058469.0,gsqofc7,1617058469.0,2019-07-01 00:00:00,personalfinance,2018-01-01 00:00:00
1,DaveAlot,ultrawidemasterrace,1615528553.0,gqnvkp0,1615528553.0,2019-07-01 00:00:00,personalfinance,2018-01-01 00:00:00
2,DaveAlot,ultrawidemasterrace,1614292110.0,gorg61l,1614292110.0,2019-07-01 00:00:00,personalfinance,2018-01-01 00:00:00
3,DaveAlot,ultrawidemasterrace,1614219096.0,gonosj2,1614219096.0,2019-07-01 00:00:00,personalfinance,2018-01-01 00:00:00
4,DaveAlot,tax,1612253656.0,glpizlx,1612253656.0,2019-07-01 00:00:00,personalfinance,2018-01-01 00:00:00


In [112]:
## get "expert" distribution
# get all posts written before question date
# compute % posted in question subreddit UGH
from tqdm import tqdm
tqdm.pandas()
def get_subreddit_pct_per_author_per_date(data, start_date):
#     print(f'data = \n{data.head()}')
#     print(f'N = {data.shape[0]}')
#     print(f'subreddit counts {data.loc[:, "subreddit"].value_counts()}')
    data.drop_duplicates('id', inplace=True)
    valid_data = data[data.loc[:, 'date_day_bin'] <= start_date]
    subreddit = data.loc[:, 'question_subreddit'].iloc[0]
    subreddit_pct = (data.loc[:, 'subreddit']==subreddit).sum() / data.shape[0]
    return subreddit_pct

In [ ]:
from tqdm import tqdm
author_subreddit_counts = []
for subreddit_i, data_i in author_question_data.groupby(['question_subreddit']):
    print(f'**** processing data from subreddit={subreddit_i} ****')
    author_subreddit_counts_i = []
    for (author_j, date_j), data_j in tqdm(data_i.groupby(['author', 'question_date_day_bin'])):
        author_subreddit_counts_j = get_subreddit_pct_per_author_per_date(data_j, data_j.loc[:, 'question_date_day_bin'].iloc[0])
        author_subreddit_counts_i.append([author_j, date_j, author_subreddit_counts_j])
    author_subreddit_counts_i = pd.DataFrame(author_subreddit_counts_i, columns=['author', 'date', 'subreddit_pct'])
    author_subreddit_counts_i = author_subreddit_counts_i.assign(**{'subreddit' : subreddit_i})
    author_subreddit_counts.append(author_subreddit_counts_i)
author_subreddit_counts = pd.concat(author_subreddit_counts, axis=0)

**** processing data from subreddit=Advice ****


100%|██████████| 27868/27868 [01:25<00:00, 326.22it/s]


**** processing data from subreddit=AmItheAsshole ****


  4%|▎         | 2346/62629 [00:16<02:56, 340.77it/s]

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
for subreddit_i, data_i in author_subreddit_counts.groupby('subreddit'):
    plt.hist(data_i.loc[:, 'subreddit_pct'].values)
    plt.title(f'% posts in subreddit={subreddit_i}')
    plt.show()

We see that the "expert" category is very sparse; can we expand this with nearest-neighbor subreddits?

In [100]:
import re
import os
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
embed_data_dir = '../../data/reddit_data/author_data/'
subreddit_embed_file_matcher = re.compile('subreddit_embeddings_.*gz')
subreddit_embed_files = list(filter(lambda x: subreddit_embed_file_matcher.match(x) is not None, os.listdir(embed_data_dir)))
subreddit_embed_files = list(map(lambda x: os.path.join(embed_data_dir, x), subreddit_embed_files))
subreddit_nearest_neighbors = []
subreddits_to_query = author_question_data.loc[:, 'question_subreddit'].unique()
top_k = 10
date_fmt = '%Y-%m-%d'
for subreddit_embed_file_i in subreddit_embed_files:
    subreddit_embed_date_i = os.path.basename(subreddit_embed_file_i).split('.')[0].split('_')[-1]
    subreddit_embed_date_i = datetime.strptime(subreddit_embed_date_i, date_fmt)
    subreddit_embed_i = pd.read_csv(subreddit_embed_file_i, sep='\t', compression='gzip', index_col=0)
    for subreddit_j in subreddits_to_query:
        # find nearest neighbors
        if(subreddit_j in subreddit_embed_i.index):
            embed_sim_j = cosine_similarity(subreddit_embed_i.loc[[subreddit_j], :].values,
                                            Y=subreddit_embed_i.values)[0, :]
            embed_sim_j = pd.Series(embed_sim_j, index=subreddit_embed_i.index).sort_values(ascending=False, inplace=False)
            top_k_neighbors_j = embed_sim_j.index.tolist()[1:(top_k+1)]
            subreddit_nearest_neighbors.append([subreddit_j, subreddit_embed_date_i, top_k_neighbors_j])
subreddit_nearest_neighbors = pd.DataFrame(subreddit_nearest_neighbors, columns=['subreddit', 'date', 'neighbors'])

In [101]:
pd.set_option('display.max_colwidth', 500)
subreddit_nearest_neighbors.sort_values('subreddit', inplace=True)
display(subreddit_nearest_neighbors)

,subreddit,date,neighbors
3,Advice,2019-01-01,"[Needafriend, Psychic, needadvice, SuicideWatch, mentalhealth, getdisciplined, Rateme, amiugly, Anxiety, rant]"
8,Advice,2018-01-01,"[dating_advice, socialskills, NoFap, AskDocs, Aquariums, SuicideWatch, Austin, confession, runescape, tipofmytongue]"
17,Advice,2019-07-01,"[answers, Needafriend, MakeNewFriendsHere, lawofattraction, Instagram, NoFap, amiugly, SuicideWatch, medical, ask]"
12,Advice,2018-07-01,"[MakeNewFriendsHere, SuicideWatch, rant, socialskills, Marriage, ApplyingToCollege, amiugly, Anxiety, solotravel, stopdrinking]"
18,AmItheAsshole,2019-07-01,"[PurplePillDebate, survivor, 6thForm, EntitledBitch, kpop, discgolf, pettyrevenge, BravoRealHousewives, notliketheothergirls, blackladies]"
4,AmItheAsshole,2019-01-01,"[IsItBullshit, asktransgender, ftm, bisexual, newzealand, pettyrevenge, LetsNotMeet, melbourne, astrology, britishproblems]"
13,AmItheAsshole,2018-07-01,"[unpopularopinion, TooAfraidToAsk, TrueOffMyChest, offmychest, AMA, rant, AskDocs, TalesFromYourServer, casualiama, changemyview]"
2,legaladvice,2019-01-01,"[Insurance, legaladviceofftopic, bestoflegaladvice, LawSchool, sanfrancisco, Scams, LegalAdviceUK, ProtectAndServe, Divorce, chicago]"
16,legaladvice,2019-07-01,"[legaladvicecanada, Insurance, legaladviceofftopic, LawSchool, lawschooladmissions, polyamory, Landlord, AskHR, Veterans, bestoflegaladvice]"
7,legaladvice,2018-01-01,"[bestoflegaladvice, LawSchool, Aquariums, askcarsales, Accounting, JUSTNOMIL, exmormon, Austin, Seattle, bicycling]"


In [108]:
## filtered data
valid_subreddits = {
    'Advice' : {'needadvice', 'mentalhealth', 'getdisciplined', 'dating_advice', 'socialskills', 'AskDocs', 'tipofmytongue', 'answers', 'ask'},
    'AmItheAsshole' : {'IsItBullshit', 'TooAfraidToAsk', 'AskDocs'},
    'legaladvice' : {'Insurance', 'bestoflegaladvice', 'LawSchool', 'Scams', 'LegalAdviceUK', 'Landlord', 'AskHR'},
    'pcmasterrace' : {'watercooling', 'overclocking', 'bapcsalescanada', 'linuxmasterrace', 'Monitors', 'SuggestALaptop', 'pcgaming', 'buildmeapc'},
    'personalfinance' : {'StudentLoans', 'CreditCards', 'churning', 'tax', 'whatcarshouldIbuy', 'ynab', 'askcarsales', 'Accounting', 'RealEstate', 'financialindependence', 'investing', 'FinancialPlanning', 'smallbusiness'},
}
from functools import reduce
subreddit_nearest_neighbors = subreddit_nearest_neighbors.assign(**{
    'valid_neighbors' : subreddit_nearest_neighbors.apply(lambda x: set(x.loc['neighbors']) & valid_subreddits[x.loc['subreddit']], axis=1)
})
## combine into one set per subreddit
subreddit_combined_neighbors = subreddit_nearest_neighbors.groupby('subreddit').apply(lambda x: set(reduce(lambda a,b: a|b, x.loc[:, 'valid_neighbors'].values))).to_dict()
print(subreddit_combined_neighbors)

{'Advice': {'socialskills', 'mentalhealth', 'needadvice', 'ask', 'answers', 'tipofmytongue', 'getdisciplined', 'AskDocs', 'dating_advice'}, 'AmItheAsshole': {'IsItBullshit', 'AskDocs', 'TooAfraidToAsk'}, 'legaladvice': {'AskHR', 'LawSchool', 'LegalAdviceUK', 'Insurance', 'bestoflegaladvice', 'Scams', 'Landlord'}, 'pcmasterrace': {'SuggestALaptop', 'buildmeapc', 'overclocking', 'watercooling', 'linuxmasterrace', 'pcgaming', 'Monitors', 'bapcsalescanada'}, 'personalfinance': {'smallbusiness', 'askcarsales', 'ynab', 'FinancialPlanning', 'churning', 'RealEstate', 'whatcarshouldIbuy', 'StudentLoans', 'CreditCards', 'investing', 'financialindependence', 'Accounting', 'tax'}}


Now that we have the nearest neighbors, let's re-run the "expert" tests.

In [109]:
def get_subreddit_neighbors_pct_per_author_per_date(data, start_date, neighbors):
    data.drop_duplicates('id', inplace=True)
    valid_data = data[data.loc[:, 'date_day_bin'] <= start_date]
    subreddit = data.loc[:, 'question_subreddit'].iloc[0]
    subreddit_neighbors = neighbors[subreddit]
    subreddit_pct = (data.loc[:, 'subreddit'].apply(lambda x: x==subreddit or x in subreddit_neighbors)).sum() / data.shape[0]
    return subreddit_pct

In [ ]:
from tqdm import tqdm
author_subreddit_counts = []
for subreddit_i, data_i in author_question_data.groupby(['question_subreddit']):
    print(f'**** processing data from subreddit={subreddit_i} ****')
    author_subreddit_counts_i = []
    for (author_j, date_j), data_j in tqdm(data_i.groupby(['author', 'question_date_day_bin'])):
        author_subreddit_counts_j = get_subreddit_neighbors_pct_per_author_per_date(data_j, data_j.loc[:, 'question_date_day_bin'].iloc[0], subreddit_combined_neighbors)
        author_subreddit_counts_i.append([author_j, date_j, author_subreddit_counts_j])
    author_subreddit_counts_i = pd.DataFrame(author_subreddit_counts_i, columns=['author', 'date', 'subreddit_pct'])
    author_subreddit_counts_i = author_subreddit_counts_i.assign(**{'subreddit' : subreddit_i})
    author_subreddit_counts.append(author_subreddit_counts_i)
author_subreddit_counts = pd.concat(author_subreddit_counts, axis=0)

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
for subreddit_i, data_i in author_subreddit_counts.groupby('subreddit'):
    plt.hist(data_i.loc[:, 'subreddit_pct'].values)
    plt.title(f'% posts in subreddit={subreddit_i}')
    plt.show()

## Expand location
We have previously identified an author's location based on whether they disclose their location in prior comments ("I live in ___").

Let's expand this by looking at location-specific subreddits.

In [ ]:
## naive approach: identify all possible locations in subreddit names by geolocation lol

